In [ ]:
!pip install transformers    # Primero instalamos la librería Transformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 57.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00


In [ ]:
# Importamos AutoTpkenizer, que nos permitirá recuperar el método de tokenización
# utilizado durante el proceso de entrenamiento de cada modelo pre-entrenado, para
# ahora poder aplicarlo en el nuevo texto de entrada:

from transformers import AutoTokenizer

In [ ]:
# Por ejemplo, el tokenizador utilizado durante el entrenamiento de gpt2:

tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [ ]:
encoding = tokenizer("This is my first sentence in the Tranformer World!", "Hello World")
print(encoding)

{'input_ids': [1212, 318, 616, 717, 6827, 287, 262, 833, 272, 16354, 2159, 0, 15496, 2159], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


#**BERT**

In [ ]:
# O bien, el tokenizer de una de las versiones de BERT ...

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
encoding = tokenizer("one two 3 4", "Altogether now")

print(encoding)

print(tokenizer.decode(encoding["input_ids"]))  # podemos recuperar el texto a través de los enteros de la tokenización

print(encoding["token_type_ids"])    # Segment Embeddings

print(tokenizer.convert_ids_to_tokens(encoding["input_ids"]))   # WordPiece Tokenizer

{'input_ids': [101, 1141, 1160, 124, 125, 102, 17762, 16609, 4679, 1208, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] one two 3 4 [SEP] Altogether now [SEP]
[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
['[CLS]', 'one', 'two', '3', '4', '[SEP]', 'Alto', '##get', '##her', 'now', '[SEP]']


In [ ]:
encoding = tokenizer("Tranformers World!", "My name is BERT.")

print(encoding)

print(tokenizer.decode(encoding["input_ids"]))

print(encoding["token_type_ids"])

print(tokenizer.convert_ids_to_tokens(encoding["input_ids"]))   # WordPiece Tokenizer

{'input_ids': [101, 157, 4047, 23763, 1116, 1291, 106, 102, 1422, 1271, 1110, 139, 9637, 1942, 119, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[CLS] Tranformers World! [SEP] My name is BERT. [SEP]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]
['[CLS]', 'T', '##ran', '##former', '##s', 'World', '!', '[SEP]', 'My', 'name', 'is', 'B', '##ER', '##T', '.', '[SEP]']


Observa que tenemos un diccionario en el cual nos regresa los siguiente:

*   **input_ids:** Los enteros de la tokenización de los enunciados de entrada.

*   **token_type_ids (segment embeddings):** En este caso que tenemos dos oraciones, nos identifica cada una.

*   **attention_mask:** Nos idica cuáles tokens necesitarán el concepto de Attention.



Observa que en particular tenemos dos tokens especiales, CLS y SEP, llamados "classifier" y "separator".

El uso de estos tokens especiales varía de acuerdo al tipo de Transformer.

#**Padding**

Cuando tenemos por ejemplo varios enunciados/comentarios/twitters de entrada de diferente longitud cada uno, se requiere usar el concepto de "padding" para que todos queden iguales en longitud de tokenizers.

Observa que se hace el padding con respecto al enunciado más largo:*texto en cursiva*

In [ ]:
txt = ["1 2 3", "Hello World", "token"]

encoding = tokenizer(txt, padding=True)

print(encoding["input_ids"])

print(encoding["attention_mask"])

[[101, 122, 123, 124, 102], [101, 8667, 1291, 102, 0], [101, 22559, 102, 0, 0]]
[[1, 1, 1, 1, 1], [1, 1, 1, 1, 0], [1, 1, 1, 0, 0]]


#**Truncation**

Puedes indicar la longitud máximo de un segmento (incluyendo los tokens especiales).

In [ ]:
txt = ["1 2 3 4 5 6 7", "a b c"]

encoding = tokenizer(txt, padding=True,
                     truncation=True,
                     max_length=6
                     )

print(encoding['input_ids'])

[[0, 134, 132, 155, 204, 2], [0, 102, 741, 740, 2, 1]]


#**Modelos**

##**Método - 1: AutoModel - Sentiment Analysis**

In [ ]:
model_name = "siebert/sentiment-roberta-large-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
my_sentences = ["This is a wonderful world", "This product was awful", "It was a good experience"]

In [ ]:
# Loading the tokenizer:

encoding = tokenizer(my_sentences,
                     padding=True,
                     truncation=True,
                     return_tensors='pt',   # probabilities
                     )
print(encoding)

{'input_ids': tensor([[    0,   713,    16,    10,  4613,   232,     2],
        [    0,   713,  1152,    21, 11522,     2,     1],
        [    0,   243,    21,    10,   205,   676,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# loading the model:

from transformers import AutoModelForSequenceClassification

In [ ]:
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)  # recuerda que cada vez que cargas algún modelo sube algo la RAM

In [ ]:
# veamos las probabilidads del sentimiento asignado a cada frase:

pt_outputs = pt_model(**encoding)
logits = pt_outputs.logits
print(logits)

tensor([[-3.6391,  2.7987],
        [ 3.9579, -3.6338],
        [-3.8043,  2.9754]], grad_fn=<AddmmBackward0>)


In [ ]:
# veamos la salida aplicando una softmax:

import torch
output = torch.softmax(logits, dim=1).tolist()

print(output)

[[0.0015973389381542802, 0.9984026551246643], [0.9994956254959106, 0.0005043695564381778], [0.0011352396104484797, 0.9988647699356079]]


##**Método - 2: Pipeline**

Podemos hacer uso de la clase Pipeline para llegar a los resultados anteriores de manera directa:

In [ ]:
from transformers import pipeline

In [ ]:
!pip install xformers

In [ ]:
# creamos una isntancia de pipeline con el modelo y tokenizador que seleccionaste:

roberta_pipe = pipeline("sentiment-analysis",
                        model= model_name,
                        tokenizer= model_name,
                        top_k=None   # para que regrese todos los scores
                        )


In [ ]:
# y de nuevo llegamos a la misma clasificación de cada enunciado:

roberta_pipe(my_sentences)

[[{'label': 'POSITIVE', 'score': 0.9984026551246643},
  {'label': 'NEGATIVE', 'score': 0.001597340451553464}],
 [{'label': 'NEGATIVE', 'score': 0.9994956254959106},
  {'label': 'POSITIVE', 'score': 0.0005043696146458387}],
 [{'label': 'POSITIVE', 'score': 0.9988647699356079},
  {'label': 'NEGATIVE', 'score': 0.0011352396104484797}]]